![](./figures/Logo.PNG)

## In this part of the tutorial, you will
- study and discuss the HBV model and its input parameters
- manually fit the model to the runoff in three different catchments
- use the residual to compare calibrated model results

---

# 1d - The HBV model

---

## 1. About HBV

The **HBV model** (Hydrologiska Byråns Vattenbalansavdelning, see [Seibert and Vis, 2012](https://hess.copernicus.org/articles/16/3315/2012/hess-16-3315-2012.pdf)) is a conceptual rainfall-runoff model widely used for hydrological forecasting and water resource management. It simulates the transformation of precipitation into runoff through several storage components, accounting for snow accumulation, soil moisture, groundwater, and river discharge.

<center>
<img src="https://vt-hydroinformatics.github.io/Quarto_Book/images/HBV-schem-Shrestha-Solomantine-2008.png" style="width:66%">
</center>

**Structure of the HBV Model**

The HBV model consists of several components that each represent fluxes of water in a hydrological system:

1. **Precipitation** Input into the model, which may be either in the form of rain or snow.
2. **Snow Accumulation and Melt** A submodel calculates snow accumulation and melt based on daily temperatures. It decides whether precipitation occurs as rainfall (RF) or snow (SF) and how much water is stored in the snow pack (SP).
3. **Soil Moisture** Describes the water storage in the unsaturated zone (SM). Controls the infiltration into this layer (I), as well as the loss through evaporation (EV).
4. **Groundwater** Represents groundwater storage in a two-bucket model. The upper fast-responding zone (UZ) is recharged from the unsaturated zone, while over time, groundwater percolates (PERC) into the slower-responding lower zone.
5. **Runoff** The flow generated from the different storages (Q) with a fast component (Q0) and a quick component (Q1)

The HBV model requires a set of semi-empirical parameters that need to be tuned for every catchment.

| **Parameter** | **Description**                                      | **Units**   | **Min Value** | **Max Value** |
|---------------|------------------------------------------------------|-------------|---------------|---------------|
| **Ts**        | Temperature threshold for snow/rain separation       | °C          | -3            | 3             |
| **CFMAX**     | Degree-day factor controlling snowmelt               | mm/°C/day   | 0             | 20            |
| **CFR**       | Coefficient for runoff response                       | -           | 0             | 1             |
| **CWH**       | Coefficient for water holding capacity                | -           | 0             | 0.8           |
| **BETA**      | Shape parameter for runoff generation                 | -           | 0             | 7             |
| **LP**        | Threshold for reduced evapotranspiration             | -           | 0.3           | 1             |
| **FC**        | Maximum soil moisture storage capacity                 | mm          | 1             | 2000          |
| **PERC**      | Percolation from upper to lower groundwater           | mm/day      | 0             | 100           |
| **K0**        | Recession coefficient for upper groundwater            | day⁻¹       | 0.05          | 2             |
| **K1**        | Recession coefficient for lower groundwater            | day⁻¹       | 0.01          | 1             |
| **K2**        | Recession coefficient for base flow                    | day⁻¹       | 0.005         | 0.1           |
| **UZL**       | Threshold for upper groundwater outflow                | mm          | 0             | 100           |
| **MAXBAS**    | Maximum base flow duration                             | days        | 1             | 6             |



<div style="background:#e0f2fe; padding:1%; border:1mm solid SkyBlue">
    <h4><span>&#129300 </span>Your Turn I: Understanding HBV</h4>
    Discuss the following points together:
    <ol>
        <li>What assumptions are implied by the HBV model? In other words: how is reality simplified and what processes are considered important or not?</li>
        <li>Looking at the parameter table: what changes in model behavior do you suspect when you change their values?</li>
    </ol>
</div>

WRITE YOUR ANSWERS HERE...

<div style="background:#e0f2fe; padding:1%; border:1mm solid SkyBlue"></div>

## 2. Using HBV

**Import packages**

In [1]:
import sys
sys.path.append('src/')
import HBV
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdate
import numpy as np
import pandas as pd
from ipywidgets import interact, Dropdown, FloatSlider

**Create and display dropdown menu for selecting catchment**

In [2]:
# DO NOT ALTER! code to select the catchment

catchment_names = ["Medina River, TX, USA", "Siletz River, OR, USA", "Trout River, BC, Canada"]
dropdown = Dropdown(
    options=catchment_names,
    value=catchment_names[0],
    description='Catchment:',
    disabled=False)

display(dropdown)

Dropdown(description='Catchment:', options=('Medina River, TX, USA', 'Siletz River, OR, USA', 'Trout River, BC…

**Some notes on pandas dataframes**: 

DataFrames are 2-dimensional data structures with columns, like a spreadsheet or SQL table. They allow quick and easy operations on the columns. Lets say we have a dataframe called df_example with two columns: 'A' and 'B'. All rows in 'A' contain the integer 1, all rows in 'B' have the integer 2. Then you can easily sum them up, creating a column 'C': 
```python
    df_example['C'] = df_example['A'] + df_example['B']
```
If there was an additional column 'D' containing the numbers of 1 to 10, you could kick out all rows of df_example where 'D' has 1s or 2s by:
```python
    df_example = df_example[df_example['D'] > 2]
```
Also, there are plenty of functionalities implemented in the pandas package, like: [mean](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html), [median](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.median.html), [max](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.max.html), [pow](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pow.html). In the following example, you will [merge](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html) two dataframes.

**Read catchment data, prepare input for model**

In [3]:
# Read catchment data
catchment_name = dropdown.value
# Read catchment data
file_dic = {catchment_names[0]: "camels_08178880", catchment_names[1]: "camels_14305500", catchment_names[2]: "hysets_10BE007"}
df = pd.read_csv(f"data/{file_dic[catchment_name]}.csv")
# Make sure the date is interpreted as a datetime object -> makes temporal operations easier
df.date = pd.to_datetime(df['date'], format='%Y-%m-%d')
# Index frame by date
df.set_index('date', inplace=True)
# Select only the columns we need
df = df[["total_precipitation_sum","potential_evaporation_sum","streamflow", "temperature_2m_mean"]]
# Rename variables
df.columns = ["P [mm/day]", "PET [mm/day]", "Q [mm/day]", "T [C]"]
# Select time frame
start_date = '2002-10-01'
end_date = '2003-09-30'
df = df[start_date:end_date]
# Reformat the date for plotting
df["Date"] = df.index.map(lambda s: s.strftime('%b-%d-%y'))
df = df.reset_index(drop=True)

# Prepare the data intput for both models
P = df["P [mm/day]"].to_numpy()
evap = df["PET [mm/day]"].to_numpy()
temp = df["T [C]"].to_numpy()

**Run HBV interactively**

In [4]:
# DO NOT ALTER! Code to interactively tune HBV model

param_names = ["Ts", "CFMAX", "CFR", "CWH", "BETA", "LP", "FC", "PERC", "K0", "K1", "K2", "UZL", "MAXBAS"]
lower = [-3, 0, 0, 0, 0, 0.3, 1, 0, 0.05, 0.01, 0.005, 0, 1] # lower bounds for the parameters
upper = [3, 20, 1, 0.8, 7, 1, 2000, 100, 2, 1, 0.1, 100, 6]  # upper bounds for the parameters 

def run_and_plot_hbv(**params):
    
    # Run HBV snow routine
    params = list(params.values())
    p_s, _, _ = HBV.snow_routine(params[:4], temp, P)
    # Run HBV runoff simulation
    Case = 1 # for now we assume that the preferred path in the upper zone is runoff (Case = 1), it can be set to percolation (Case = 2)
    ini = np.array([0,0,0]) # initial state
    Q_sim, _, _ = HBV.hbv_sim(params[4:], p_s, evap, Case, ini)
    
    # Prepare plot of results
    fig, ax = plt.subplots(figsize=(20, 4))  # set figure size

    # Plot the simulated and observed runoff (Q)
    ax.plot(df.index, Q_sim, label="HBV")
    sns.lineplot(data=df, x="Date", y="Q [mm/day]", color="black", label="Observed")

    # Show only the main ticks
    locator = mdate.MonthLocator()
    plt.gca().xaxis.set_major_locator(locator)

    ax.set_title(catchment_name)
    
    # Display the figure
    plt.show()

params = {param: FloatSlider(min=xmin, max=xmax, step=(xmax - xmin)/10, value=(xmax + xmin)/2, description=param) for param, xmin, xmax in zip(param_names, lower, upper)}
interact(run_and_plot_hbv, **params);

interactive(children=(FloatSlider(value=0.0, description='Ts', max=3.0, min=-3.0, step=0.6), FloatSlider(value…

<div style="background:#e0f2fe; padding:1%; border:1mm solid SkyBlue">
    <h4><span>&#129300 </span>Your Turn II: Manually Fit the Model</h4>
    <ol>
        <li>Try to manually alter the parameters to get a good fit.
            <ul>
                <li>Which induce a strong change in the hydrograph?</li>
                <li>Which are important for a good-quality fit?</li>
            </ul>
        </li>
        <li>Discuss, how the model captures different features of the hydrograph (e.g., flow volume, low flows, peak amplitude, peak timing) and where it fails to do so.</li>
    </ol>
    <i>You may want to note down the parameter values for each catchment in the table below, as you will need them later.</i>
</div>

| Catchment             | TT  | CFMAX | FC  | LP  | BETA | K0   | K1   | K2   | PERC | UZL  | MAXBAS |
|-----------------------|-----|-------|-----|-----|------|------|------|------|------|------|--------|
| Medina River, TX, USA  | ?   | ?     | ?   | ?   | ?    | ?    | ?    | ?    | ?    | ?    | ? |
| Siletz River, OR, USA  | ?   | ?     | ?   | ?   | ?    | ?    | ?    | ?    | ?    | ?    | ? |
| Trout River, BC, Canada| ?   | ?     | ?   | ?   | ?    | ?    | ?    | ?    | ?    | ?    | ? |

<div style="background:#e0f2fe; padding:1%; border:1mm solid SkyBlue">
    <h4><span>&#129300 </span>Your Turn III: Look at the Model Residuals</h4>
    <p>Once you are satisfied with your manual calibration results: use the python cell below to take a closer look at the residuals. 
        <ol>
            <li>What insights about the model can you gain by looking at them?</li>
        </ol>
    </p>
    <p>Use statistical metrics you know (mean, standard deviation, ...) to help you discuss the following questions:</p>
    <ol start=2>
        <li>How good is the flow volume reflected?</li>
        <li>Are low flows captured well?</li>
        <li>Compared to other calibrations (e.g. your neighbour's): which simulation result is closer to the observed values?</li>
    </ol>
    <i>Before you start: remove the "'''" at the top and bottom, and make sure you enter the parameter values from above.<i>
</div>

**Computing and visualizing the residuals**

In [5]:
"""
########### Set parameters ##################
# Replace these placeholders with the values you found earlier
# Check that you loaded the correct catchment

#TT =    # TODO  # Temperature threshold (°C)
#CFMAX = # TODO  # Degree-day factor (mm/°C/day)
#FC =    # TODO  # Field capacity (mm)
#LP =    # TODO  # Wilting point (0 to 1)
#BETA =  # TODO  # Shape parameter for runoff generation
#K0 =    # TODO  # Recession coefficient for upper groundwater (1/day)
#K1 =    # TODO  # Recession coefficient for lower groundwater (1/day)
#K2 =    # TODO  # Recession coefficient for base flow (1/day)
#PERC =  # TODO  # Percolation (mm/day)
#UZL =   # TODO  # Threshold for upper groundwater outflow (mm)

############################################

# Run HBV snow routine
#params = [TT, CFMAX, FC, LP, BETA, K0, K1, K2, PERC, UZL]  # Collecting all parameters
p_s, _, _ = HBV.snow_routine(params[:4], temp, P)  # Use first 4 parameters for snow routine

# Run HBV runoff simulation
Case = 1  # Set to 1 for runoff, 2 for percolation
ini = np.array([0, 0, 0])  # Initial state (upper zone, lower zone, groundwater)
Q_sim, states, fluxes = HBV.hbv_sim(params[4:], p_s, evap, Case, ini)

# combine results into dataframe
df_model = pd.DataFrame({"Q_obs [mm/day]": df.iloc[:,2].to_numpy(), "Q_sim [mm/day]": Q_sim, "ET [mm/day]": fluxes.T[0]}, index=df.Date)

########### code below this line ##################

# analyse the residual. Be creative! Sum up / take mean or median / filter observed values (analyse only what you are interested in)

df_model["residuals"] = # TODO

# plot the residuals
"""
pass